In [1]:
import sys
import os
import torch
sys.path.append("C:/Users/yuuta/Documents/fashion")
from utils.util import filter_basic_items, open_json
from utils.util import is_target_category
from utils.util import calculate_euclid_sum
from utils.infer import id_to_vector
from utils.util import calc_roc_auc
from utils.util import calc_cw_score
from utils.util import scale_versability, scale_compatibility
import json
import glob
import random
import pandas as pd
from utils.util import get_category


c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\yuuta\anaconda3\envs\fashion\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [30]:
import importlib
import utils
importlib.reload(utils.util)
from utils.util import calc_cw_score
from utils.util import scale_versability, scale_compatibility

In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda")  # GPUデバイスを取得
else:
    device = torch.device("cpu")  # CPUデバイスを取得

In [4]:
filepaths = glob.glob(
    f"C:/Users/yuuta/Documents/fashion/data/test/**/*.json"
)

In [5]:
id_to_vector_versatility = open_json('C:/Users/yuuta/Documents/fashion/experiments/data/id_to_vector_versatility.json')
id_to_vector_compatibility = open_json('C:/Users/yuuta/Documents/fashion/experiments/data/id_to_vector_compatibility.json')

In [6]:
items = {"tops": [], "bottoms": [], "shoes": []}
for fp in filepaths:
    json_dict = open_json(fp)
    if json_dict == None:
        continue
    for item in filter_basic_items(json_dict["items"]):
        category = get_category(item)
        if category == None:
            continue
        items[category].append(str(item['itemId']))

In [7]:
categories = ['tops', 'bottoms', 'shoes']
# 枚数
T = 3

In [31]:
com_score = []
ver_score = []
N = 50000
for i in range(N):
    if i % 100 == 0:
        print(f'\r {i * 100 / N}%', end='')
    dataset = {
        'tops':random.sample(items["tops"], 10),
        'bottoms':random.sample(items["bottoms"], 10),
        'shoes':random.sample(items["shoes"], 10),
    }

    # cwの初期化
    cw = {
        'tops':random.sample(dataset["tops"], T),
        'bottoms':random.sample(dataset["bottoms"], T),
        'shoes':random.sample(dataset["shoes"], T),
    }
                    
    c, v = calc_cw_score(cw, id_to_vector_versatility, id_to_vector_compatibility)
    # print('======')
    # print(c, v)
    # c, v = scale_compatibility(v), scale_versability(v)
    # print(c, v)
    # print('======')
    com_score.append(c)
    ver_score.append(v)

 99.8%

In [32]:
df = pd.DataFrame({'compatibiltity': com_score, 'versatility':ver_score})

print(df.describe())

       compatibiltity   versatility
count    50000.000000  50000.000000
mean         0.315186      0.307970
std          0.105947      0.133647
min          0.010347      0.002577
25%          0.237198      0.206186
50%          0.309805      0.296392
75%          0.384801      0.394330
max          0.973879      0.920103


In [22]:
com = df.describe()['compatibiltity']
ver = df.describe()['versatility']

In [25]:
print(
    f'min_value = {com["min"]}\nmax_value = {com["max"]}\n'
)

min_value = 7.839434299071509e-05
max_value = 0.0003497860890687236



In [26]:
print(
    f'min_value = {ver["min"]}\nmax_value = {ver["max"]}\n'
)

min_value = 642.0
max_value = 1030.0

